# 07wk-1: (합성곱신경망) – CNN 자랑, CNN 핵심레이어

최규빈  
2025-04-16

<a href="https://colab.research.google.com/github/guebin/DL2025/blob/main/posts/06wk-2.ipynb"><img src="https://colab.research.google.com/assets/colab-badge.svg" style="text-align: left"></a>

# 1. 강의영상

In [ ]:
# {{<video https://youtu.be/playlist?list=PLQqh36zP38-wcPiCEdYML9-6-Xv5RVbso&si=BbNo6mwCHqwOV0FS>}}

# 2. Imports

In [2]:
import torch
import torchvision
import matplotlib.pyplot as plt

In [3]:
plt.rcParams['figure.figsize'] = (4.5, 3.0)

# 3. CNN 자랑

## A. 성능좋음

*Fashion MNIST*

In [47]:
train_dataset = torchvision.datasets.FashionMNIST(root='./data', train=True, download=True)
test_dataset = torchvision.datasets.FashionMNIST(root='./data', train=False, download=True)
train_dataset = torch.utils.data.Subset(train_dataset, range(5000))
test_dataset = torch.utils.data.Subset(test_dataset, range(1000))
to_tensor = torchvision.transforms.ToTensor()
X = torch.stack([to_tensor(img) for img, lbl in train_dataset]).to("cuda:0")
y = torch.tensor([lbl for img, lbl in train_dataset])
y = torch.nn.functional.one_hot(y).float().to("cuda:0")
XX = torch.stack([to_tensor(img) for img, lbl in test_dataset]).to("cuda:0")
yy = torch.tensor([lbl for img, lbl in test_dataset])
yy = torch.nn.functional.one_hot(yy).float().to("cuda:0")

*발악수준으로 설계한 신경망*

In [56]:
net = torch.nn.Sequential(
    torch.nn.Flatten(),
    torch.nn.Linear(784,2048),
    torch.nn.ReLU(),
    torch.nn.Linear(2048,10)
).to("cuda")
loss_fn = torch.nn.CrossEntropyLoss()
optimizr = torch.optim.Adam(net.parameters())

In [49]:
for epoc in range(1,500):
    #1
    logits = net(X)
    #2
    loss = loss_fn(logits, y) 
    #3
    loss.backward()
    #4 
    optimizr.step()
    optimizr.zero_grad()

In [50]:
(net(X).argmax(axis=1) == y.argmax(axis=1)).float().mean()

In [51]:
(net(XX).argmax(axis=1) == yy.argmax(axis=1)).float().mean()

*대충대충 설계한 합성곱신경망*

In [52]:
net = torch.nn.Sequential(
    torch.nn.Conv2d(1,16,2),
    torch.nn.ReLU(),
    torch.nn.MaxPool2d(2),
    torch.nn.Flatten(),
    torch.nn.Linear(2704,10),
).to("cuda")
loss_fn = torch.nn.CrossEntropyLoss()
optimizr = torch.optim.Adam(net.parameters())

In [53]:
for epoc in range(1,500):
    #1
    logits = net(X)
    #2
    loss = loss_fn(logits, y) 
    #3
    loss.backward()
    #4 
    optimizr.step()
    optimizr.zero_grad()

In [54]:
(net(X).argmax(axis=1) == y.argmax(axis=1)).float().mean()

In [55]:
(net(XX).argmax(axis=1) == yy.argmax(axis=1)).float().mean()

## B. 파라메터적음

In [76]:
net1 = torch.nn.Sequential(
    torch.nn.Flatten(),
    torch.nn.Linear(784,2048),
    torch.nn.ReLU(),
    torch.nn.Linear(2048,10)
)
net2 = torch.nn.Sequential(
    torch.nn.Conv2d(1,16,2),
    torch.nn.ReLU(),
    torch.nn.MaxPool2d(2),
    torch.nn.Flatten(),
    torch.nn.Linear(2704,10),
)

In [77]:
net1_params = list(net1.parameters())
net2_params = list(net2.parameters())

In [78]:
for params in net1_params:
    print(params.shape)

torch.Size([2048, 784])
torch.Size([2048])
torch.Size([10, 2048])
torch.Size([10])

In [79]:
2048*784 + 2048 + 10*2048 + 10 

In [80]:
for params in net2_params:
    print(params.shape)

torch.Size([16, 1, 2, 2])
torch.Size([16])
torch.Size([10, 2704])
torch.Size([10])

In [81]:
16*1*2*2 + 16 + 10*2704 + 10

## C. 유명함

`-` <https://brunch.co.kr/@hvnpoet/109>

# 4. CNN 핵심레이어

## A. `torch.nn.Conv2d`

`-` 우선 연산하는 방법만 살펴보자.

**(예시1)**

In [12]:
torch.manual_seed(43052)
conv = torch.nn.Conv2d(1,1,(2,2)) # 입력1, 출력1, (2,2) window size
conv.weight.data, conv.bias.data

In [13]:
img = torch.arange(0,4).reshape(1,1,2,2).float() # 2,2 흑백이미지. 
img

In [14]:
(-0.1733)*0 + (-0.4235)*1 +\
(0.1802)*2 + (0.4668)*3 + 0.2037

In [15]:
conv(img)

**(예시2) 평균**

In [46]:
conv = torch.nn.Conv2d(1,1,(2,2))
conv.weight.data = conv.weight.data * 0 + 1/4
conv.bias.data = conv.bias.data * 0

In [47]:
img

In [48]:
conv(img)

**(예시3) 이동평균?**

In [101]:
img = torch.arange(16).reshape(1,1,4,4).float()
img,conv(img)

**(예시4) window size가 증가한다면? (2d의 이동평균느낌)**

In [95]:
img = torch.arange(16).reshape(1,1,4,4).float()
conv = torch.nn.Conv2d(1,1,(3,3))
conv.weight.data = conv.weight.data * 0 + 1/9
conv.bias.data = conv.bias.data * 0

In [96]:
img,conv(img)

**(예시5)** 2개의 이미지

In [97]:
imgs = torch.arange(32).reshape(2,1,4,4).float()
conv = torch.nn.Conv2d(1,1,(3,3))
conv.weight.data = conv.weight.data * 0 + 1/9
conv.bias.data = conv.bias.data * 0

In [98]:
imgs, conv(imgs)

**(예시6) 피처뻥튀기**

In [35]:
imgs = torch.arange(32).reshape(2,1,4,4).float()
conv = torch.nn.Conv2d(1,16,(3,3))

In [36]:
imgs.shape,conv(imgs).shape

> 질문: `conv(img) = img @ What` 을 만족하는 What을 찾을 수 있을까?

## B. `torch.nn.ReLU`

In [38]:
relu = torch.nn.ReLU()
img = torch.randn(25).reshape(1,1,5,5)
img, relu(img)

## C. `torch.nn.MaxPool2d`

In [39]:
mp = torch.nn.MaxPool2d((2,2))
img = torch.randn(25).reshape(1,1,5,5)
img,mp(img)